In [ ]:
import pandas as pd
import numpy as np
import json

from funciones_varias import desanidarColumna, desanidarVariasColumnas, creaDfFuncionesFastAPI

#cargo los datasets
#si agrego  dtype='unicode' evito algunos errores sobre tipos de datos
dfMovies = pd.read_csv(r'datasets_crudos\movies_dataset.csv', dtype='unicode')
dfCredits= pd.read_csv(r'datasets_crudos\credits.csv', dtype='unicode')

In [ ]:
#TRANSFORMACIONES VARIAS ---> NO OLVIDAR CORRER LO QUE ESTA COMENTADO

#lleno NaNs con 0s en las columnas revenue y budget
#elimino rows que contienen un NaN en la columna release_date
#elimino columnas que no se utilizaran
try:
    dfMovies['revenue'] = dfMovies['revenue'].fillna(0)
    dfMovies['budget'] = dfMovies['budget'].fillna(0)
    dfMovies = dfMovies.dropna(subset='release_date')
    dfMovies= dfMovies.drop(columns=["video","adult","imdb_id" ,"original_title", "poster_path", "homepage"])

    #elimino estas rows que tienen los valores mezclados y hacen que me de error despues
    dfMovies.drop(dfMovies[dfMovies["id"]=="1997-08-20"].index, inplace=True)
    dfMovies.drop(dfMovies[dfMovies["id"]=="2014-01-01"].index, inplace=True)
    dfMovies.drop(dfMovies[dfMovies["id"]=="2012-09-29"].index, inplace=True)
    
except:
    print("estas transformaciones ya se ejecutaron")


In [ ]:
#desanido belongs to collection
franquicia = (desanidarColumna(dfMovies['belongs_to_collection']))

#inserto nueva columna y elimino la vieja
dfMovies= dfMovies.drop(columns=["belongs_to_collection"])
dfMovies.insert(1, "franquicia", franquicia["name"])

In [14]:
#la columna generos hay que desanidarle cada columna despues de aplciarle la funcion

dfGeneros = (desanidarColumna(dfMovies['genres'])) #le aplico la primera funcion
dfGeneros=desanidarVariasColumnas(dfGeneros) #desanido cada columna con la segunda funcion      
dfGeneros=dfGeneros.iloc[:45376] #esto es cualquier cosa pero bueno, funciona xd
                                 #uso iloc para obtener solo hasta el row 45378 porque sino tengo errores
                                 
#inserto nueva columna y elimino la vieja
dfMovies.insert(1, "movie_genre", dfGeneros["name"])
dfMovies= dfMovies.drop(columns=["genres"])

In [15]:
#desanido tabla production_companies
dfCompanies=desanidarColumna(dfMovies['production_companies'])
dfCompanies=desanidarVariasColumnas(dfCompanies)
dfCompanies= dfCompanies.iloc[:45376] #REVISAR COMO HACER MEJOR ESTA PARTE

#inserto nueva columna y elimino la vieja
dfMovies.insert(7, "production_company", dfCompanies["name"])
dfMovies= dfMovies.drop(columns=["production_companies"])

In [16]:
#desanido production_countries
dfProdCountries=desanidarVariasColumnas(desanidarColumna(dfMovies["production_countries"]))
dfProdCountries = dfProdCountries.iloc[:45376] #REVISAR COMO HACER MEJOR ESTA PARTE

#inserto nueva columna y elimino la vieja
dfMovies.insert(8, "production_country", dfProdCountries["name"])
dfMovies= dfMovies.drop(columns=["production_countries"])

In [17]:
#desanido spoken_languages
dfSpokenLan=desanidarVariasColumnas(desanidarColumna(dfMovies["spoken_languages"]))
dfSpokenLan = dfSpokenLan.iloc[:45376] #REVISAR COMO HACER MEJOR ESTA PARTE

#inserto nueva columna y elimino la vieja
dfMovies.insert(12, "language", dfSpokenLan["name"]) #puedo elegir entre poner la abreviacion o la palabra entera
dfMovies= dfMovies.drop(columns=["spoken_languages"])

In [18]:

#tablas que contienen fechas:
    #release_date
release_year = dfMovies["release_date"]

#obtengo solo año de cada string de fecha y lo meto en release_year
release_year = pd.to_datetime(release_year).dt.year

#inserto nueva columan release_year
dfMovies.insert(10, "release_year", release_year)

#se supone que transforme la columna en date type pero es un int REVISAR

In [19]:
#comienzo a armar columna return

#paso a float
dfMovies["revenue"]=pd.DataFrame(dfMovies["revenue"]).astype(float)
dfMovies["budget"]=pd.DataFrame(dfMovies["budget"]).astype(float)

#creo la columna return y relleno con 0 cuando el resultado es None
dfMovies["return"]=round(dfMovies["revenue"]/dfMovies["budget"], 2)
dfMovies['return'] = dfMovies['return'].fillna(0)

#reemplazo valores infinitos
dfMovies.replace([np.inf, -np.inf], 0, inplace=True)

#para revisar si hay valores infinitos
counter=0
for i in dfMovies["return"]:
    if i == float("inf"):
        print(i)
        counter+=1
print(counter)



0


In [20]:
#desanido la columna crew y la dejo en un df separado (tarda como 13 minutos en armarse)

dfCrew = pd.DataFrame()
for i in range(len(dfCredits)):
    stringsCrew = dfCredits.crew[i]
    dictionaryCrew = eval(stringsCrew) #esto es para pasarlo de string a lista y poder armar el df
    dictionaryCrew = pd.DataFrame(dictionaryCrew)
    dictionaryCrew["id"] = dfCredits.id[i]  #reemplazo el id del df con el de dfCredits para poder unirlo a su pelicula
    dfCrew = pd.concat([dfCrew, dictionaryCrew])
    #dfCrew = dfCrew.reset_index(drop=True) #si descomento el reindex tarda mas del doble

dfCrew.to_csv(r"C:\Users\\Administrator\\Desktop\\dfCrew.csv")

In [21]:
#desanido la columna cast y la dejo en un df separado (tarda como 17 minutos en armarse)
dfCast = pd.DataFrame()
for i in range(len(dfCredits)):
    stringsCast = dfCredits.cast[i]
    dictionaryCast = eval(stringsCast) #esto es para pasarlo de string a lista y poder armar el df
    dictionaryCast = pd.DataFrame(dictionaryCast)
    dictionaryCast["id"] = dfCredits.id[i] #reemplazo el id del df con el de dfCredits para poder unirlo a su pelicula
    dfCast = pd.concat([dfCast, dictionaryCast])
    #dfCast = dfCast.reset_index(drop=True) #si descomento el reindex tarda mas del doble

dfCast.to_csv(r"C:\Users\\Administrator\\Desktop\\dfCast.csv")

In [22]:
#cambio de posicion la columna "title" y le cambio el nombre a "movie_title"
titles=pd.DataFrame(dfMovies["title"])
dfMovies.insert(0,"movie_title" ,titles)
dfMovies=dfMovies.drop(columns=["title"])

In [23]:
#considero que para poder continuar con el modelo de clasificacion, debo poner el tipo de dato que corresponde a cada columna

strings= ["movie_title", "movie_genre", "original_language", "overview", "production_company",
           "language", "franquicia", "production_country"]
integers=["release_year", "vote_count"]
floats=["runtime","return", "budget", "vote_average", "revenue", "popularity"]

#dropeo valor raro porque sino no puedo ponerle el data type que corresponde
dfMovies.drop(dfMovies[dfMovies['popularity'] == 'Beware Of Frost Bites'].index, inplace=True)

for i in strings:
    dfMovies[i] = dfMovies[i].astype("string")
for i in integers:
    dfMovies[i] = dfMovies[i].astype("int")
for i in floats:
    dfMovies[i].replace("", 0, inplace=True) #hay strings vacios en vote_count por eso los lleno con 0
    dfMovies[i] = dfMovies[i].astype("float")
dfMovies["release_date"] = pd.to_datetime(dfMovies["release_date"])

dfMovies.dtypes

movie_title           string[python]
budget                       float64
movie_genre           string[python]
franquicia            string[python]
id                            object
original_language     string[python]
overview              string[python]
popularity                   float64
production_company    string[python]
production_country    string[python]
release_date          datetime64[ns]
release_year                   int32
revenue                      float64
runtime                      float64
language              string[python]
status                        object
tagline                       object
vote_average                 float64
vote_count                     int32
return                       float64
dtype: object

In [24]:
#para exportar csv e ir probando cargarlo a la API
dfMovies.to_csv(r"C:\Users\\Administrator\\Desktop\\dfMovies.csv")


In [25]:
#PRIMERA FUNCION DATAFRAME dfPeliculasIdiomas

#dataset funcion peliculas idioma
dfPeliculasIdiomas=creaDfFuncionesFastAPI(["movie_title", "language", "original_language"], dfMovies)

#guardo dataset para la primera funcion
dfPeliculasIdiomas.to_csv(r"C:\Users\\Administrator\\Desktop\\dfPeliculasIdiomas.csv")

In [26]:
#SEGUNDA FUNCION DATAFRAME dfMoviesDuration

#nuevo df con la info que necesito para la segunda funcion
dfMoviesDuration=creaDfFuncionesFastAPI(["movie_title", "release_year", "runtime"], dfMovies)

#guardo dataset para la segunda funcion
dfMoviesDuration.to_csv(r"C:\Users\\Administrator\\Desktop\\dfMoviesDuration.csv")

In [27]:
#TERCERA FUNCION DATAFRAME dfFranquicia

#POR EL MOMENTO, GANANCIA TOTAL VA A SER GIUAL A REVENUE
#dfMovies["ganancia_total"]=round(dfMovies["revenue"]-dfMovies["budget"], 2)

pd.options.display.float_format = '{:.2f}'.format #porque sino me tira numeros con notacion cientifica

#solo obtengo 5381 registros ya que hay muchos campos nulos tanto en budget como en revenue
#dfMovies.loc[(dfMovies["revenue"] > 0) & (dfMovies["budget"] > 0) , ["movie_title","franquicia","budget","revenue", "ganancia_total"]]

#la funcion toma como parametro FRANQUICIA
#necesito CANTIDAD de peliculas de esa franquicia
#GANACIA total (revenue)
#PROMEDIO dela ganancia total

#filtro revenue mayor a 0 y me quedo con columnas que necesito
dfFiltrado = dfMovies.loc[(dfMovies["revenue"] > 0), ["franquicia","movie_title","revenue"]]

#hago group by por franquicia y agrego cada una de las maneras para agrupar las demas columnas
dfFranquicia = dfFiltrado.groupby("franquicia").agg(
    movie_count=("movie_title", "count"),
    ganancia_total=("revenue", "sum"),
    ganancia_promedio=("revenue", "mean")).reset_index()

#guardo dataset para la tercera funcion
dfFranquicia.to_csv(r"C:\Users\\Administrator\\Desktop\\dfFranquicia.csv")




In [28]:
#CUARTA FUNCION DATAFRAME dfPaises

#def peliculas_pais( Pais: str ): Se ingresa un país (como están escritos en el dataset, no hay que traducirlos),
# retornando la cantidad de peliculas producidas en el mismo.
#    Ejemplo de retorno: Se produjeron X películas en el país X

#creo df para la cuarta funcion
dfPaises = creaDfFuncionesFastAPI(["production_country", "movie_title"], dfMovies)

#agrupo por pais y cuento las peliculas
dfPaises = dfPaises.groupby("production_country").agg(
    movie_count=("movie_title", "count")).reset_index()

#exporto df para cuarta funcion
dfPaises.to_csv(r"C:\Users\\Administrator\\Desktop\\dfPaises.csv")


In [29]:
#QUINTA FUNCION DATAFRAME dfProductora

#def productoras_exitosas( Productora: str ): Se ingresa la productora, entregandote el revunue total y la
# cantidad de peliculas que realizo.
#Ejemplo de retorno: La productora X ha tenido un revenue de x

dfProductora = creaDfFuncionesFastAPI(["production_company", "movie_title","revenue"], dfMovies)

dfProductora = dfMovies.loc[(dfMovies["revenue"] > 0), ["production_company","movie_title","revenue"]]

dfProductora = dfProductora.groupby("production_company").agg(
    movie_count =("movie_title", "count"),
    ganancia_total = ("revenue", "sum")
).reset_index()

#exporto df para quinta funcion
dfProductora.to_csv(r"C:\Users\\Administrator\\Desktop\\dfProductora.csv")

In [30]:
#armo dfs que voy a usar para la sexta funcion

dfMovieReturn = creaDfFuncionesFastAPI(["id", "return"], dfMovies)

dfMovieInfo = creaDfFuncionesFastAPI(["id", "movie_title","release_date", "budget","revenue","return"], dfMovies)

dfDirectores = dfCrew.loc[(dfCrew["job"] == "Director"), ["id","job","name"]]

In [31]:
#SEXTA FUNCION DATAFRAME 1 dfDirectoresExito

#filtro para no traer los returns en 0
dfMovieReturn = dfMovieReturn.loc[dfMovieReturn["return"]>0]

#para revisar cuantos directores hay por pelicula
#dfDirectoresPorPelicula = dfDirectores.groupby("id").agg(
#    job_count=("job", "count"),
#    name_count=("name", "count")
#).reset_index()
#dfDirectoresPorPelicula[dfDirectoresPorPelicula["job_count"]>1]

#hago join para poder calcular el return de cada director
dfDirectoresExito = dfDirectores.merge(dfMovieReturn, on="id",how="left")

#dropeo columnas que no me sirven
dfDirectoresExito=dfDirectoresExito.drop(columns=["job", "id"])

#hago group by director name
dfDirectoresExito = dfDirectoresExito.groupby("name").agg(
    director_return = ("return", "mean")
)

#resultado sin tener en cuenta cuando no hay return diponible para el director
dfDirectoresExito = dfDirectoresExito.loc[dfDirectoresExito["director_return"]>0]

#exporto uno de los dfs que voy a usar para la sexta funcion
dfDirectoresExito.to_csv(r"C:\Users\\Administrator\\Desktop\\dfDirectoresExito.csv")

In [32]:
#SEXTA FUNCION DATAFRAME 2 dfDirectoresInfoPeliculas

#filtro para no traer valores en 0
dfMovieInfo = dfMovieInfo.loc[dfMovieInfo["return"]>0]
#hago join para poder calcular el return de cada director
dfDirectoresInfoPeliculas = dfMovieInfo.merge(dfDirectores, on="id",how="left")

#hago reindex
dfDirectoresInfoPeliculas = dfDirectoresInfoPeliculas.reindex(columns=["id","name", "job","movie_title", "release_date",
                                                                "budget", "revenue", "return"])
#dropeo columnas que no me sirven
dfDirectoresInfoPeliculas=dfDirectoresInfoPeliculas.drop(columns=["job", "id"])                                                                 

#exporto el otro de los dfs que voy a usar para la sexta funcion
dfDirectoresInfoPeliculas.to_csv(r"C:\Users\\Administrator\\Desktop\\dfDirectoresInfoPeliculas.csv")   